In [3]:
import copy
import numpy as np
import periodictable as pip
import re
import os
import sys


nameout=str(sys.argv[1])
# path=str(sys.argv[2])
path="/scratch/mam/QM_Part/H20_diss/str_opt/B3LYP_def2_TZVP/rigid_scan/freq_scan/xaa_f/xaa_f.out"
fileout=open(nameout+".out","w+")     



print(""" Entering Gaussian System

 Cite this work as:
 Gaussian 16, Revision A.03,


 Cite this work as:
 Gaussian 16, Revision A.03,

 
 *********************************************
 Gaussian 98:  ??? No! this is from TURBOMOLE
 
 ********************************************* 

 
                         Standard orientation:                        
 ---------------------------------------------------------------------
 Center     Atomic     Atomic              Coordinates (Angstroms)
 Number     Number      Type              X           Y           Z
 ---------------------------------------------------------------------""", file=fileout)






# path =  "/home/mam/Praktikum_TC/QM_Part/H20_diss/str_opt/B3LYP_def2_TZVP/rigid_scan/freq_scan/xaa_f/xaa_f.out" 


#set path through console / as current directory


############################################################################################
# FUNCTIONS #

def reader(searcher, checker=False):

    with open(path,"r") as orcafile:
        for line in orcafile:                      #reads line in file and converts section into list
            if searcher in line:
                checker=True
                tmp=[]
            if checker==True:
                if line !="\n":
                    tmp.append(line.split())
                else:
                    checker=False
    return tmp


def cutter(listtocut,rowno,listend,liststart=0):     #cuts rows of given list
    newlist=[]
    listcp=copy.deepcopy(listtocut)
    for i in range(liststart,listend):
        tmp=listcp[i][rowno]
        newlist.append(tmp)
    return newlist


def ncutter(listtocut,listend,liststart=0):          #preps normal mode datalist for easier use later
    newlist=[]
    listcp=copy.deepcopy(listtocut)
    for i in range(liststart,listend):
        tmpcutter=listcp[i]
        newlist.append(tmpcutter)
    del newlist[::int(n)+1]
    
    for i in range(0,len(newlist)):
        del newlist[i][0]
    del newlist[0:int(n)]
    return newlist


def floater(listofstrings,rangestart=0,rowno=0,offset=0,decimals="{:.4f}"):     #decimals input in the form of "{:.4f}"
    tmpfloater=[]                                                            #converts string data into float with format of choice
    listcp=copy.deepcopy(listofstrings)
    for i in range(rangestart,len(listcp)):
        el = listcp[i+offset][rowno]
        el = float(el)
        el = decimals.format(el)                            
        tmpfloater.append(el)
        if i == len(listofstrings):
            break
    return tmpfloater


def coordcutter(listtocut,coord):                  #takes every third coordinate of the normal mode list
    listcp=copy.deepcopy(listtocut)                #del for offset, so either x,y or z will be taken
    if coord=="y":
        del listcp[0]

    elif coord=="z":
        del listcp[0]
        del listcp[0]
    elif coord=="x":
        listcp=listcp
    tmpcoord=listcp[::3]
    return tmpcoord


def ncutter2(liste):                              #takes the multicolumn list for normal modes and converts it into one column list
    tmpliste=[]                                   #also checks for the width of the last "block" of coordinates
    width=6
    widthlast=6
    listecp=copy.deepcopy(liste)
    blocks=int(len(listecp)/n)
    
    
    if len(listecp[-1])==1:
        widthlast=1
    elif len(listecp[-1])==2:
        widthlast=2
    elif len(listecp[-1])==3:
        widthlast=3
    elif len(listecp[-1])==4:
        widthlast=4
    elif len(listecp[-1])==5:
        widthlast=5
    
    for k in range(0,blocks-1):
        for j in range(0,width):

            for i in range((k)*n,((k+1))*n):

                el=listecp[i][j]
                el=float(el)
                el="{:.2f}".format(el)
                tmpliste.append(el)
        
    for k in range(blocks-1,blocks):
        for j in range(0,widthlast):

            for i in range((k)*n,((k+1))*n):

                el=listecp[i][j]
                el=float(el)
                el="{:.2f}".format(el)
                tmpliste.append(el)
        
    return tmpliste


def rot_temp(list_rotational_const):         #converts the rotational constants data into floats
    tmp=[]                      #calculates the rotational temperature for the given rotational constants
    del list_rotational_const[0][0:4]
    tmp.append(list_rotational_const[0])
    for i in range(len(tmp[0])):
        tmp[0][i]=float(tmp[0][i])
    tmp2=[]
    for i in range(len(tmp[0])):      
        tmp2.append(tmp[0][i]*6.626*10**(-34)*300000000/(1.38065*10**(-23))*100)
        tmp2[i]="{:.5f}".format(tmp2[i])
    return tmp2


#FUNCTIONS
###############################################################################################

inpcoords=reader("CARTESIAN COORDINATES (ANGSTROEM)")        #input coordinates of the atoms   
atomno=reader("CARTESIAN COORDINATES (A.U.)")                #list containing atomic numbers
            
lastL1 = False                                               #checks for end of list
 
ANl=floater(atomno,3,2,0,"{:.0f}")                          #atomic numbers

Xinp=floater(inpcoords,2,1,0,"{:.6f}")
Yinp=floater(inpcoords,2,2,0,"{:.6f}")
Zinp=floater(inpcoords,2,3,0,"{:.6f}")


for i  in range(len(ANl)):
        j = str(-1 + i)

        
        print(f" {str(j):>4}{str(ANl[i]):>11}             0{str(Xinp[i]):>16}{str(Yinp[i]):>12}{str(Zinp[i]):>12}", file=fileout) #    1          8             0        0.000000     0.000000   0.000000
        if inpcoords[i] == inpcoords[-1]:
            lastL1 = True
        if lastL1 == True:
            print(" ---------------------------------------------------------------------", file=fileout)
        if i == len(inpcoords):
            break

print(" ---------------------------------------------------------------------",file=fileout)





nmode=reader("                  0          1          2          3          4          5    ") #list containing coordinates to normal modes
n=int(nmode[-1][0])+1                       #length of each block of normal mode displacements (=3N)

nmode_cut=ncutter(nmode,len(nmode))

     
numbers=[]
for i in range(n):                          #1,2,3,4,...n
    k=1
    numbers.append(i+k)

    
nnmode_cut=int(nmode[-1][0])+1
    
coord=ncutter2(nmode_cut)

coordsx=coordcutter(coord,"x")
coordsy=coordcutter(coord,"y")
coordsz=coordcutter(coord,"z")



colnumber=[]                               #number of the column
for i in range(len(ANl)*3-6):
    
    k=1
    k=k+i
    colnumber.append(k)

    

freq=reader("   0:")                      #list containing frequencies


fq = floater(freq,0,1)                                   #list of frequencies
del fq[0:6]                                              #deletes first 6 (all 0)


ir=reader(" Mode   freq       eps      Int      T**2         TX        TY        TZ")

IR = floater(ir,3,2)                                  #list of IR-intesities
        
                  
        
###########################################################################################################  
for i in range(len(fq)):                #Position Wsl nicht richtig,weil listen unterschiedlich lang!!!!!!
    if float(fq[i])<0:
        IR.insert(i,"0.0000") 
############################################################################################################
    
    
freqamount=int(len(coordsx)/n)


for j in range(0,freqamount):

    
    print(f"{str(colnumber[3*j]):>23}",end="", file=fileout)
    print(f"{str(colnumber[3*j+1]):>23}",end="", file=fileout)
    print(f"{str(colnumber[3*j+2]):>23}", file=fileout)
    print("                      A                      A                      A  ", file=fileout)

    print(f" Frequencies --{fq[3*j]: >11}{fq[3*j+1]: >23}{fq[3*j+2]: >23}", file=fileout)
    print(" Red. masses --     0.0000                 0.0000                 0.0000            ", file=fileout)  
    print(" Frc consts  --     0.0000                 0.0000                 0.0000            ", file=fileout)
    print(f" IR Inten    --{IR[3*j]: >11}{IR[3*j+1]: >23}{IR[3*j+2]: >23}", file=fileout)
    print(" Raman Activ --     0.0000                 0.0000                 0.0000            ", file=fileout)
    print(" Depolar     --     0.0000                 0.0000                 0.0000            ", file=fileout)
    
    
    
    print(""" Atom   AN      X      Y      Z        X      Y      Z        X      Y      Z""", file=fileout)
    
    for i in range(0,len(ANl)):             

        print(f"{str(numbers[i]): >5}",end="", file=fileout)
        print(f"{str(ANl[i]): >5}",end="", file=fileout)

        print(f"{str(coordsx[i+j*n]): >9}",end="", file=fileout)
        print(f"{str(coordsy[i+j*n]): >7}",end="", file=fileout)
        print(f"{str(coordsz[i+j*n]): >7}",end="", file=fileout)

        print(f"{str(coordsx[i+len(ANl)+j*n]): >9}",end="", file=fileout)
        print(f"{str(coordsy[i+len(ANl)+j*n]): >7}",end="", file=fileout)
        print(f"{str(coordsz[i+len(ANl)+j*n]): >7}",end="", file=fileout)

        print(f"{str(coordsx[i+len(ANl)*2+j*n]): >9}",end="", file=fileout)
        print(f"{str(coordsy[i+len(ANl)*2+j*n]): >7}",end="", file=fileout)
        print(f"{str(coordsz[i+len(ANl)*2+j*n]): >7}", file=fileout)



        if i==len(ANl):
            break



inp=reader("""*xyzfile""")                                       #searches for input data in orca file

totalm=reader("""Total Mass""")                                  #searches for total mass


molecularmassf = float(totalm[0][3])
molecularmass = "{:.6f}".format(molecularmassf)

rotc=reader("""Rotational constants in MHz :""")                #seaches for rotational constants

                
rotationalc = rotc[0][5:]

for i in range(0,len(rotationalc)):
    rotationalc[i]=float(rotationalc[i])
    rotationalc[i]=rotationalc[i]*0.001
    rotationalc[i]="{:.6f}".format(rotationalc[i])
    
    if i==len(rotationalc):
        break
        
print(f"Charge =  "+str(inp[0][3])+"  Multiplicity = "+str(inp[0][4]), file=fileout)
print("Molecular mass:  "  +str(molecularmass)+"  amu.", file=fileout)
print(f"Rotational constants (GHZ):{str(rotationalc[0]): >17}{str(rotationalc[1]): >15}{str(rotationalc[2]): >15}", file=fileout)




SCF = reader("Total Energy       :")                           #searches for last SCF value



SCFdone = SCF[0][3]   
SCFdone = float(SCFdone)
SCFdone = "{:.6f}".format(SCFdone)
 
    
functional=reader("|  1>")                      #looks for used functional in orca file


funct=functional[0][3]     
            
ZeroPcorr=reader("Total correction")            #searches for zero-point correction energy

    
ZeroPointCorr=ZeroPcorr[0][2]
ZeroPointCorr=float(ZeroPointCorr)
ZeroPointCorr="{:.6f}".format(ZeroPointCorr)
    


print(f"SCF Done:  E(R{str(funct):>0}) ={str(SCFdone): >12}", file=fileout)                       
print(f"Zero-point correction={str(ZeroPointCorr) :>9} (Hartree/Particle)", file=fileout) 

elmass =[]

for i in range(0,len(ANl)):
    for el in pip.elements: 

        if str(el.number) == ANl[i]:
            elmass.insert(len(ANl), el.mass)
            elmass[i] = float(elmass[i])
            elmass[i] = "{:.6f}".format(elmass[i])
            print(f"AtmWgt={str(elmass[i]): >13}", file=fileout)
        if i == len(ANl):
            break
print("Normal termination of Gaussian 16", file=fileout)

rot_c=reader("Rotational constants in cm-1:     ")

rotational_temp=rot_temp(rot_c)
print(f" Rotational temperature (Kelvin) {rotational_temp[0]:>13}{rotational_temp[1]:>12}{rotational_temp[2]:>12}",file=fileout)


fileout.close()